In [1]:
!pip install apache-tvm
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 30.4 MB/s eta 0:00:00:00:0100:01


In [2]:
# !sudo apt install nvidia-cuda-toolkit -y
# !conda install -c conda-forge cudatoolkit=11.2.2 cudnn=8.1.0 -y


In [3]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/valeradanger/miniconda3/pkgs/cudatoolkit-11.2.2-hbe64b41_10/lib/
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/x86_64-linux-gnu

In [4]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [5]:
# !pip install "tensorflow<2.11" 

In [6]:
# !pip install tensorflow[and-cuda]==2.12

In [7]:
!pip install tensorflow==2.12 keras==2.12 matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 52.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
  Attempting uninstall: gast
    Found existing installation: g

In [2]:
import tvm
from tvm import relay
from tvm.contrib import graph_executor
import os
import time
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt # plot the first image in the dataset
import keras
import tensorflow as tf
from keras.applications.resnet import ResNet50
from tvm import auto_scheduler

In [3]:
from tensorflow.keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input

In [4]:
import os
#n01632777 акселотли
folder_path = "/kaggle/input/imagenet-validation-dataset/imagenet_validation/n01632777"
image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('.jpg', '.png', '.jpeg'))]
print(image_files)

['ILSVRC2012_val_00046976.JPEG', 'ILSVRC2012_val_00011474.JPEG', 'ILSVRC2012_val_00041298.JPEG', 'ILSVRC2012_val_00022113.JPEG', 'ILSVRC2012_val_00037046.JPEG', 'ILSVRC2012_val_00003860.JPEG', 'ILSVRC2012_val_00013940.JPEG', 'ILSVRC2012_val_00006694.JPEG', 'ILSVRC2012_val_00017274.JPEG', 'ILSVRC2012_val_00022771.JPEG', 'ILSVRC2012_val_00007486.JPEG', 'ILSVRC2012_val_00035529.JPEG', 'ILSVRC2012_val_00016514.JPEG', 'ILSVRC2012_val_00004084.JPEG', 'ILSVRC2012_val_00005172.JPEG', 'ILSVRC2012_val_00039922.JPEG', 'ILSVRC2012_val_00025438.JPEG', 'ILSVRC2012_val_00016441.JPEG', 'ILSVRC2012_val_00045656.JPEG', 'ILSVRC2012_val_00022925.JPEG', 'ILSVRC2012_val_00040894.JPEG', 'ILSVRC2012_val_00047938.JPEG', 'ILSVRC2012_val_00002524.JPEG', 'ILSVRC2012_val_00017899.JPEG', 'ILSVRC2012_val_00044504.JPEG', 'ILSVRC2012_val_00020430.JPEG', 'ILSVRC2012_val_00045553.JPEG', 'ILSVRC2012_val_00036495.JPEG', 'ILSVRC2012_val_00020772.JPEG', 'ILSVRC2012_val_00006579.JPEG', 'ILSVRC2012_val_00008686.JPEG', 'ILSVRC

In [5]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
all_images = []
for img_name in image_files:
    img_path = f'/kaggle/input/imagenet-validation-dataset/imagenet_validation/n01632777/{img_name}'
    img = image.load_img(img_path,target_size=(224,224))
    x = image.img_to_array(img)
    #x = np.expand_dims(x,axis=0) #(num_samples, 224, 224, 3) нужно если по одной
    x = preprocess_input(x)
    all_images.append(x)

x_data = np.array(all_images)
print(x_data.shape)

# (x_train, y_train), (x_test, y_test) = cifar10.load_data() #(num_samples, 224, 224, 3)
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)
# x_train = x_train.reshape(50000,32,32,3)
# x_test = x_test.reshape(10000,32,32,3)

# print(x_train.shape)
# print(x_test.shape)

(50, 224, 224, 3)


In [6]:
model = ResNet50(
    weights="imagenet",
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()
print("finish")
# #model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30)
# model.save("resnet50_cifar10ep30v2.h5")
#model = keras.models.load_model("/kaggle/input/my-files/resnet50_cifar10ep30.h5")

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [7]:
#model = keras.models.load_model("/kaggle/input/resnettestnoepcifar/resnet50_cifar10ep30v2.h5")
#model.summary()
#print(x_train[:50].shape)
print("Prediction: ", np.argmax(model.predict(x_data), axis=1))
#print("Labels:     ", np.argmax(y_train[:50], axis=1))
#Замер времени
start_time = time.time()
predictions = model.predict(x_data, batch_size=1) #поправил batch_size
end_time = time.time()
inference_time_keras = end_time - start_time
print("Время инференса модели Keras: {} секунд".format(inference_time_keras))

2/2 [==============================] - 5s 1s/step
Prediction:  [  6 126 338  29  29  29  29  29  29  29  29 124  29  29  29  29  29 329
  29  29 103  35  39  29  29  29 633  29 712  29  29  29  29  29  29  29
  29  29   1  29  29  29  29  29  29  29  79  29  50  29]
50/50 [==============================] - 4s 82ms/step
Время инференса модели Keras: 4.206265926361084 секунд


In [8]:
input_shape = [1, 224, 224, 3] # [batch, height, width, channels]
#shape_dict = {"input_input": input_shape}
shape_dict = {"input_1": input_shape}
from tvm import relay
mod, params = relay.frontend.from_keras(model, shape_dict, layout="NHWC") #подгрузка модели
#print(mod)


target = tvm.target.Target("llvm  -mcpu=core-avx2")
#target = tvm.target.Target("llvm")
dev = tvm.cpu(0)

with tvm.transform.PassContext(opt_level=3): #проводим тесты над нейросетью в tvm //    tvm_model(data.reshape(1, 28, 28, 1)).numpy()[0]
    tvm_model = relay.build_module.create_executor("graph", mod, dev, target, params).evaluate()

In [9]:
print(x_data.shape)
reshaped_data = [data.reshape(1, 224, 224, 3) for data in x_data]
results = []

# Проводим инференс над измененными данными
start_time_tvm = time.time()
for data in reshaped_data:
    tvm_model(data).numpy()[0]
end_time_tvm = time.time()
inference_time_tvm = end_time_tvm - start_time_tvm
print("Время инференса модели TVM: {} секунд".format(inference_time_tvm))

(50, 224, 224, 3)
Время инференса модели TVM: 11.089477062225342 секунд
